# PyZX interoperability

We can use PyZX to load intermediate representations like QASM to simulate with PyQrack, as well as to optimize circuits with the ZX-calculus before simulating them.

In [1]:
import sys; sys.path.insert(0,'..')
import os
from collections import Counter
import pyzx as zx
from pyqrack import QrackSimulator

If the file has the expected extension and other format properties, we should be able to load any intermediate representation that PyZX knows with `pyzx.Circuit.load()`.

In [2]:
fname = os.path.join('qasm','pyzx_demo.qasm')
circ = zx.Circuit.load(fname)

Before optimizing the circuit, we need to convert it to a graph representation.

In [3]:
c = circ.to_graph()

Then, we can optimize the graph.

In [4]:
zx.simplify.full_reduce(c)

Once the graph is loaded and optionally optimized, we can feed it directly to a QrackSimulator for immediate simulation

In [5]:
sim = QrackSimulator(pyzxCircuit=circ)
Counter(sim.measure_shots(range(circ.qubits), 64))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir


Counter({0: 28, 3: 36})

Optionally, to run unitary ZX-calculus graph subroutines as parts of a larger program, we can use `QrackSimulator.run_pyzx_gates()` instead of the special constructor.

In [6]:
sim = QrackSimulator(qubitCount=2)
sim.run_pyzx_gates(circ.gates)
Counter(sim.measure_shots(range(circ.qubits), 64))

Counter({0: 35, 3: 29})